We will use data from *Bees of Guam* as a test case.

http://tb.plazi.org/GgServer/dwca/FE566D11FFD2FFF5383F9056FFE3FFEF.zip

In [1]:
import pandas as pd
#from dwca.read import DwCAReader
import os
from datetime import datetime

In [2]:
UUID = 'FE566D11FFD2FFF5383F9056FFE3FFEF'

In [3]:
def generate_html_report(results_html, summary_html):
    timestamp = datetime.utcnow()
    html = f'''
        <html>
            <header>
                <meta charset="utf-8">
                <meta name="viewport" content="width=device-width, initial-scale=1">
                <title>mat_cit_chk</title>
                <link rel="stylesheet" href="https://cdn.jsdelivr.net/npm/bulma@0.9.3/css/bulma.min.css">
            </header>
            <body>
                <section class="section">
                    <div class="container">
                        {summary_html}
                        {results_html}
                    </div>
                </section>
            </body>
        </html>        
        '''
    with open(f'{UUID}_new.html', 'w') as f:
        f.write(html)    

In [ ]:
def create_status_report():
        
    summary_list = []
    for uuid in uuids:
        summary = json.load(open(f'{uuid}_summary.json'))
        summary['uuid'] = uuid
        summary_list.append(summary)
    df_summary = pd.DataFrame(summary_list)
    df_merged = pd.merge(df, df_summary, on='uuid')
    df_merged.drop('status', axis=1, inplace=True)
    #df_merged['validation report'] = f'<a href="{df_merged.uuid}.html">validation report</a>'
    summary_html = df_merged.to_html(index=False)
    
    # Edit so that the table is nicely styled with Bulma
    
    summary_html = summary_html.replace('<table border="1" class="dataframe">', '<table class="table">')
    summary_html = summary_html.replace('<th>', '<th class="has-text-centered">')
    summary_html = summary_html.replace('<td>', '<td class="has-text-centered">')
    
    # Link uuid to validation report
    
    for uuid in uuids:
        summary_html = summary_html.replace(uuid, f'<a href="{uuid}.html">{uuid}</a>')
    
    # Highlight rows which need work
    
    for i, r in df_merged.iterrows():
        if (r['bad collectors'] + r['bad locations'] + r['bad dates']) > 0:
            summary_html = nth_repl(summary_html, '<tr', '<tr class="is-selected"', i+2)  
    
    title_html = f'''
        <p class="title is-1">Insects of Guam Datamining Project</p>
        <p class="subtitle is-3">Status report generated by GGIbot at {datetime.utcnow()} UTC</p>
        <p>Highlited documents failed validation. Click on the **uuid** to see the validation report.</p>
        '''

    html = f'''
        <html>
            <header>
                <meta charset="utf-8">
                <meta name="viewport" content="width=device-width, initial-scale=1">
                <title>mat_cit_chk</title>
                <link rel="stylesheet" href="https://cdn.jsdelivr.net/npm/bulma@0.9.3/css/bulma.min.css">
            </header>
            <body>
                <section class="section">
                    <div class="container">
                        {title_html}
                        {summary_html}
                    </div>
                </section>
            </body>
        </html>        
        '''
    with open('status_report_new.html', 'w') as f:
        f.write(html) 
        
# create_status_report()

In [4]:
%%time
dwca_url = f'http://tb.plazi.org/GgServer/dwca/{UUID}.zip'
dwca_file = f'{UUID}.zip'

# download the DwCA into the current working directory, 
# overwriting any previous DwCA with same uuid

result = os.system(f'wget -O {dwca_file} {dwca_url}')
assert(result==0)

# unzip the DwCA, overwriting files:
#   meta.xml, eml.xml, taxa.txt, occurrences.txt, multimedia.txt, description.txt, distribution.txt, 
#   media.txt, references.txt, vernaculars.txt

result = os.system(f'unzip -o {dwca_file}')
assert(result==0)

CPU times: user 614 µs, sys: 73 µs, total: 687 µs
Wall time: 526 ms


In [5]:
taxon_df = pd.read_csv('taxa.txt', sep='\t')
taxon_df.head(1)

,taxonID,namePublishedIn,acceptedNameUsageID,parentNameUsageID,originalNameUsageID,kingdom,phylum,class,order,family,...,scientificNameAuthorship,canonicalName,verbatimScientificName,basionymAuthors,basionymYear,combinationAuthors,combinationYear,taxonomicStatus,nomenclaturalStatus,references
0,026F1569FFD3FFF439229473FCE8F968.taxon,NaN,NaN,NaN,NaN,Animalia,Arthropoda,Insecta,Hymenoptera,Megachilidae,...,(Cockerell),Hylaeus guamensis,Hylaeus guamensis (Cockerell),Cockerell,NaN,NaN,NaN,NaN,NaN,http://treatment.plazi.org/id/026F1569FFD3FFF4...


In [6]:
occ_df = pd.read_csv('occurrences.txt', sep='\t')
occ_df.head(1)

,occurrenceID,taxonID,catalogNumber,collectionCode,institutionCode,typeStatus,verbatimLabel,sex,individualCount,eventDate,...,minimumElevationInMeters,maximumElevationInMeters,minimumDepthInMeters,maximumDepthInMeters,country,stateProvince,municipality,locality,references,basisOfRecord
0,026F1569FFD3FFF439229473FCE8F968.mc.3AAEAE22FF...,026F1569FFD3FFF439229473FCE8F968.taxon,NaN,NaN,NaN,NaN,"Sumay Road, June 23, Swezey.",NaN,1,1936-06-23,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Sumay Road,http://treatment.plazi.org/id/026F1569FFD3FFF4...,PreservedSpecimen


In [7]:
for i, taxon in taxon_df.iterrows():
    print(taxon.canonicalName)
    for i, occ in occ_df[occ_df.taxonID==taxon.taxonID].iterrows():
        print(f'   {occ.country}   {occ.locality}   {occ.recordedBy}   {occ.eventDate}')

Hylaeus guamensis
   nan   Sumay Road   O. H. Swezey   1936-06-23
Lithurgus guamensis
   nan   Tarague Beach   O. H. Swezey   1936-05-17
   nan   Machanao   R. L. Usinger & O. H.Swezey   1936-06-30
   nan   Orote Peninsula   O. H. Swezey   1936-09-27
   nan   Yigo   O. H. Swezey   1936-05-19
   nan   Yigo   O. H. Swezey   1936-10-18
Megachile fullawayi
Megachile laticeps
   nan   Tarague Beach   E. H. Bryan   1936-04-16
   nan   Agana   Rowley   1936-08-13
   nan   Barrigada   O. H. Swezey   1936-06
   nan   Piti   O. H. Swezey   1936-05-12
   nan   Piti   O. H. Swezey   1936-07-07
   nan   Piti   O. H. Swezey   1936-07-04
   nan   Piti   O. H. Swezey   1936-09-16
Halictus swezeyi
   nan   Ritidian Point   E. H. Bryan   1936-04-16
   nan   Mt. Sasalaguan   E. H. Bryan   1936-04-25
   nan   Yona   E. H. Bryan   1936-04-29
   nan   Dededo   O. H. Swezey   1936-05-11
   nan   Santa Rosa Peak   O. H. Swezey   1936-05-19
   nan   Merizo   O. H. Swezey & R. L. Usinger   1936-06-11
   nan   A

In [8]:
taxon_df.columns

Index(['taxonID', 'namePublishedIn', 'acceptedNameUsageID',
       'parentNameUsageID', 'originalNameUsageID', 'kingdom', 'phylum',
       'class', 'order', 'family', 'genus', 'taxonRank', 'scientificName',
       'scientificNameAuthorship', 'canonicalName', 'verbatimScientificName',
       'basionymAuthors', 'basionymYear', 'combinationAuthors',
       'combinationYear', 'taxonomicStatus', 'nomenclaturalStatus',
       'references'],
      dtype='object')

In [9]:
occ_df.columns

Index(['occurrenceID', 'taxonID', 'catalogNumber', 'collectionCode',
       'institutionCode', 'typeStatus', 'verbatimLabel', 'sex',
       'individualCount', 'eventDate', 'recordedBy', 'recordNumber',
       'decimalLatitude', 'decimalLongitude', 'minimumElevationInMeters',
       'maximumElevationInMeters', 'minimumDepthInMeters',
       'maximumDepthInMeters', 'country', 'stateProvince', 'municipality',
       'locality', 'references', 'basisOfRecord'],
      dtype='object')

In [10]:
# MAIN

results_html = '<p>results_html goes here</p>'
summary_html = '<p>summary_html goes here</p>'
generate_html_report(results_html, summary_html)